# Function to create a 3D value of type HPC representing the bone structure of a (parametric) reinforced concrete building.

I create a bone structure of a reinforced concrete building, starting to workshop numer 1.

<h2>Reference project model
![alt text](reference_model.jpg)

"""INFO: Telai separati fra loro di 4 mt, pilastri 300x40x40cm, travi 400x40x40cm, nessuna fondazione"""

Import libraries

In [41]:
from pyplasm import *
import csv

Function to create pillar and beam (develop at workshop n.1) for a single frame

In [42]:
def newStructure(beamSection, pillarSection, distancePillar, height):
    px=pillarSection[0]
    py=pillarSection[1]
    pz=height[0]
    bx=beamSection[0]
    bz=beamSection[1]
    xPillar=[]
    yPillar=[]
    zPillar=[]
    xBeam=[]
    zBeam=[]
    yBeam=[]
    lenBeam=0

#creo pilastri
    for i in distancePillar:
        yPillar=yPillar+[py,-i]
        xPillar=[px]
        lenBeam=lenBeam+i+py
    yPillar = yPillar + [py]    
#creo travi
    for i in height:
       zPillar=zPillar+[i,-bz]
       zBeam=zBeam+[-i,bz]
       xBeam=[bx]
       yBeam = [lenBeam+py]
        
    aPillar=PROD([QUOTE(yPillar),QUOTE(zPillar)])
    pillar=PROD([QUOTE(xPillar),aPillar])

    aBeam=PROD([QUOTE(yBeam),QUOTE(zBeam)])
    beam=PROD([QUOTE(xBeam),aBeam])
    
    frame=STRUCT([pillar, beam])
    return frame

Function to create orizzontal beam

In [43]:
def newOrizzontalBeam(bx,bz,px,py,distance,height):
    beamX = [bx]
    beamY = []
    beamZ = []
    
    for i in height:
        beamZ = beamZ + [-i,bz]
    for i in distance:
        beamY = beamY + [py,-i]
    beamY = beamY + [py]
    temp = PROD([QUOTE(beamX),QUOTE(beamY)])
    orizzoBeam = PROD([temp,QUOTE(beamZ)])

    orizzontalModel = STRUCT([orizzoBeam])
    return orizzontalModel

Function to open CSV file and pass the value to the old function for create a single telaio, and generate the final structure of a building

In [44]:
def ggpl_bone_structure(fileName):
    x = []
    y = []
    z = []
    
    orizzontalBeam = []
    frames = []
    trans = []
    
    structure = []
    structureSecondary = []
    Vx=0
    
    with open(fileName,'rU') as csvfile:  #read .csv file
        reader = csv.DictReader(csvfile)
        cont = 2
        for row in reader:
            if(cont%2 != 0): #se riga dispari
                beamSection = row['x'].replace("(","").replace(")","").split(";") #replace 'delete' the () and [], split divide values with ;
                pillarSection = row['y'].replace("(","").replace(")","").split(";")
                
                beamX = eval(beamSection[0]) #convert value
                beamZ = eval(beamSection[1])
                
                pillarX = eval(pillarSection[0])
                pillarY = eval(pillarSection[1])
                                         
                distancePillar = []
                for dist in row['z'].replace("[","").replace("]","").split(";"):
                    distancePillar.append(eval(dist))
                
                height = []
                for alt in row['k'].replace("[","").replace("]","").split(";"):
                    height.append(eval(alt))
                
                orizzontalBeam.append(newOrizzontalBeam(Vx,beamZ,pillarX,pillarY,distancePillar,height)) #create more beams                                                            
                frames.append(newStructure((beamX,beamZ),(pillarX,pillarY),distancePillar,height)) #create more frames
                
            if(cont%2 == 0): #se riga pari
                Vx = x = eval(row['x'])
                y = eval(row['y'])
                z = eval(row['z'])
                trans.append(T([1,2,3])([x,y,z])) #translation on coordinate x,y,z
               
            cont = cont+1
            
    cont2 = 0
    for fram in frames:
        structure.append(trans[cont2])
        structure.append(frames[cont2])
        if(cont2+1 < len(frames)):
            structureSecondary.append(trans[cont2])
            structureSecondary.append(orizzontalBeam[cont2+1])
        cont2=cont2+1
        
    firstView = STRUCT(structure) #generate the frames
    secondView = STRUCT(structureSecondary) #generate the beams for connect more frames
    VIEW(STRUCT([firstView,secondView]))
    

Launch function

In [45]:
ggpl_bone_structure("frame_data_435092.csv")

<h2>Final result!</h2>
![alt text](final_result.jpg)